In [2]:
import duckdb

con = duckdb.connect()

con.sql("""
INSTALL spatial;
LOAD spatial;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# Saving GeoJSON to DuckDB

In [14]:
import duckdb
import geojson
from ipyleaflet import Map, GeoJSON

# Connect to DuckDB and create a table
conn = duckdb.connect(':memory:')
conn.execute("CREATE TABLE geojson_data (id INTEGER PRIMARY KEY, geojson_str STRING)")

# Function to insert GeoJSON data into DuckDB
def insert_geojson(geojson_id, geojson_data):
    geojson_str = geojson.dumps(geojson_data)
    conn.execute("INSERT INTO geojson_data VALUES (?, ?)", (geojson_id, geojson_str))

# Function to retrieve GeoJSON data from DuckDB
def retrieve_geojson():
    result = conn.execute("SELECT geojson_str FROM geojson_data")
    for row in result.fetchall():
        geojson_str = row[0]
        geojson_data = geojson.loads(geojson_str)
        yield geojson_data

# Read GeoJSON from file
def read_geojson_file(file_path):
    with open(file_path, 'r') as file:
        geojson_data = geojson.load(file)
    return geojson_data

# Example GeoJSON file path
geojson_file_path = 'data/input_data/input_data/geospatial/geo_survey_OMBILI.geojson'

# Read GeoJSON data from file
example_geojson = read_geojson_file(geojson_file_path)

# Insert GeoJSON data into DuckDB
insert_geojson(1, example_geojson)

# Create a map
m = Map(center=(0, 0), zoom=2)

# Retrieve GeoJSON data from DuckDB and display it on the map
for geojson_data in retrieve_geojson():
    geojson_layer = GeoJSON(data=geojson_data)
    m.add_layer(geojson_layer)

# Display the map
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
# Saving SHP file to DuckDB

In [4]:
import duckdb
import geopandas as gpd
import geojson
from ipyleaflet import Map, GeoJSON

# Connect to DuckDB and create a table
conn = duckdb.connect(':memory:')
conn.execute("CREATE TABLE geojson_data (id INTEGER PRIMARY KEY, geojson_str STRING)")

# Function to insert GeoJSON data into DuckDB
def insert_geojson(geojson_id, geojson_data):
    geojson_str = geojson.dumps(geojson_data)
    conn.execute("INSERT INTO geojson_data VALUES (?, ?)", (geojson_id, geojson_str))

# Function to retrieve GeoJSON data from DuckDB
def retrieve_geojson():
    result = conn.execute("SELECT geojson_str FROM geojson_data")
    for row in result.fetchall():
        geojson_str = row[0]
        geojson_data = geojson.loads(geojson_str)
        yield geojson_data

# Read Shapefile and convert to GeoJSON
def read_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    # Convert GeoDataFrame to GeoJSON
    geojson_data = gdf.__geo_interface__
    return geojson_data

# Example Shapefile path
shapefile_path = "data/input_data/input_data/geospatial/Ombili.shp"

# Read Shapefile and convert to GeoJSON
example_geojson = read_shapefile(shapefile_path)

# Insert GeoJSON data into DuckDB
insert_geojson(1, example_geojson)

# Create a map
m = Map(center=(0, 0), zoom=2)

# Retrieve GeoJSON data from DuckDB and display it on the map
for geojson_data in retrieve_geojson():
    geojson_layer = GeoJSON(data=geojson_data)
    m.add_layer(geojson_layer)

# Display the map
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [5]:
import duckdb
import geopandas as gpd
from shapely import wkb
from ipyleaflet import Map, GeoJSON

# Connect to DuckDB and create a table
conn = duckdb.connect(':memory:')
conn.execute("CREATE TABLE shapefile_data (id INTEGER PRIMARY KEY, geometry BLOB, attributes STRING)")

# Function to insert Shapefile data into DuckDB
def insert_shapefile(gdf):
    for index, row in gdf.iterrows():
        geom_bytes = row.geometry.wkb
        attributes = str(row.drop('geometry').to_dict())
        conn.execute("INSERT INTO shapefile_data VALUES (?, ?, ?)", (index, geom_bytes, attributes))

# Function to retrieve Shapefile data from DuckDB
def retrieve_shapefile():
    result = conn.execute("SELECT id, geometry, attributes FROM shapefile_data")
    for row in result.fetchall():
        id = row[0]
        geom_bytes = row[1]
        attributes = row[2]
        geometry = wkb.loads(geom_bytes)
        yield id, geometry, attributes

# Example Shapefile path
shapefile_path = "data/input_data/input_data/geospatial/Ombili.shp"

# Read Shapefile
gdf = gpd.read_file(shapefile_path)

# Insert Shapefile data into DuckDB
insert_shapefile(gdf)

# Create a map
m = Map(center=(0, 0), zoom=2)

# Retrieve Shapefile data from DuckDB and display it on the map
for id, geometry, attributes in retrieve_shapefile():
    geojson_layer = GeoJSON(data={
        "type": "Feature",
        "geometry": geometry.__geo_interface__,
        "properties": {"id": id, "attributes": attributes}
    })
    m.add_layer(geojson_layer)

# Display the map
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…